In [3]:
# ORM：Object-Relational Mapping，把关系数据库的表结构映射到对象上

# # 导入:
from sqlalchemy import Column, String, create_engine,Integer
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# 创建对象的基类:
Base = declarative_base()

# 定义User对象:
class User(Base):
    # 表的名字:
    __tablename__ = 'user'
    #  表的结构:
    ## 字段类型要声明长度
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    fullname = Column(String(50))
    nickname = Column(String(50))
    def __repr__(self):
        return "<User(name='%s', id='%s')>" % (self.name, self.id)
# 初始化数据库连接:
engine = create_engine(r'sqlite:///text.sqlite',echo=True)
# 创建DBSession类型,使用时要实例化
DBSession = sessionmaker(bind=engine)
# 创建数据库表
Base.metadata.create_all(engine)
#若修改了表结构，可能需要手动去修改schema，



2020-11-21 23:24:55,668 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-21 23:24:55,669 INFO sqlalchemy.engine.base.Engine ()
2020-11-21 23:24:55,673 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-21 23:24:55,676 INFO sqlalchemy.engine.base.Engine ()
2020-11-21 23:24:55,679 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2020-11-21 23:24:55,681 INFO sqlalchemy.engine.base.Engine ()


- 插入数据

In [4]:
#1 先实例化对象
add_=User(name='lin',nickname='werido',fullname='hongji')
#2 插入数据库,此时未写入数据库,这里记得把session实例化
session=DBSession()
session.add(add_)
#3 写入数据库
session.commit()

2020-11-21 23:24:55,702 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-21 23:24:55,705 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 23:24:55,707 INFO sqlalchemy.engine.base.Engine ('lin', 'hongji', 'werido')
2020-11-21 23:24:55,714 INFO sqlalchemy.engine.base.Engine COMMIT


- 插入多条，补充说明


In [5]:
session.add_all([
  User(name='lin1',nickname='werido',fullname='hongji'),
  User(name='lin2',nickname='werido',fullname='hongji'),
  User(name='lin3',nickname='werido',fullname='hongji')
])
##  添加后未写入数据库，但是可以查询得到,commit才会写入数据库
lin=session.query(User).filter_by(name='lin').first()
lin_all=session.query(User).filter_by(name='lin1').all()
print(lin,lin_all)

## 展示事物中存在的未commit的记录
session.new

2020-11-21 23:24:55,879 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-21 23:24:55,882 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 23:24:55,885 INFO sqlalchemy.engine.base.Engine ('lin1', 'hongji', 'werido')
2020-11-21 23:24:55,898 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 23:24:55,906 INFO sqlalchemy.engine.base.Engine ('lin2', 'hongji', 'werido')
2020-11-21 23:24:55,910 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 23:24:55,918 INFO sqlalchemy.engine.base.Engine ('lin3', 'hongji', 'werido')
2020-11-21 23:24:55,928 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2020-11-21 23:24:55,943 INFO sqlalchemy.engine.base.Engine ('lin', 1, 0)
2020

IdentitySet([])

- 修改数据

In [6]:
lin.name='lin_mend'
session.dirty
# 已经修改过，会记录在session.dirty 里面提交后会清除记录
session.commit()

2020-11-21 23:24:56,022 INFO sqlalchemy.engine.base.Engine UPDATE user SET name=? WHERE user.id = ?
2020-11-21 23:24:56,026 INFO sqlalchemy.engine.base.Engine ('lin_mend', 2)
2020-11-21 23:24:56,032 INFO sqlalchemy.engine.base.Engine COMMIT


- session.rollback():回滚到上次/commit 状态

In [17]:
### 查询
for instance in session.query(User).order_by(User.id):
    print(instance.name)

## 参数个数要对应
for name, fullname in session.query(User.name, User.fullname):
    print(name,fullname)

for row in session.query(User, User.name).all():
     print(row.User,row.name)

## select name as lin_mend
for row in session.query(User.name.label('lin_mend')).all():
     print(row.lin_mend)

## 多条件查询
for user in session.query(User).filter(User.name=='lin_mend').filter(User.fullname=='hongji'):
 print(user)



2020-11-21 23:50:23,075 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user ORDER BY user.id
2020-11-21 23:50:23,080 INFO sqlalchemy.engine.base.Engine ()
lin_mend
lin_mend
lin
lin1
lin2
lin1
lin2
lin1
lin2
lin1
lin2
lin3
lin
lin1
lin2
lin3
2020-11-21 23:50:23,088 INFO sqlalchemy.engine.base.Engine SELECT user.name AS user_name, user.fullname AS user_fullname 
FROM user
2020-11-21 23:50:23,098 INFO sqlalchemy.engine.base.Engine ()
lin_mend hongji
lin_mend hongji
lin hongji
lin1 hongji
lin2 hongji
lin1 hongji
lin2 hongji
lin1 hongji
lin2 hongji
lin1 hongji
lin2 hongji
lin3 hongji
lin hongji
lin1 hongji
lin2 hongji
lin3 hongji
2020-11-21 23:50:23,103 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user
2020-11-21 23:50:23,104 INFO sqlalchemy.engine.base.Engine ()
<User(name='l

In [ ]:
## 使用原生SQL语句
